In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import gc
samplerows=10000
os.getcwd()

# To check about the 2019 weekly file path df week filter for the rolling later

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Tracker_By_Migration_JT'

In [2]:
prior_quarter_End=datetime.date(2019,5,4)
recent_complete_quarter_End=prior_quarter_End+datetime.timedelta(days=7*13)

print("prior_quarter_End",prior_quarter_End)
print("recent_complete_quarter_End",recent_complete_quarter_End)

lapsed_date_begin=prior_quarter_End-datetime.timedelta(days=(4*365+1+6))
print("lapsed_date_begin",lapsed_date_begin)


prior_quarter_End 2019-05-04
recent_complete_quarter_End 2019-08-03
lapsed_date_begin 2015-04-28


In [3]:
lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str).drop_duplicates()
lapsed=lapsed[lapsed['transaction_date']>=str(lapsed_date_begin)]

print(lapsed['transaction_date'].min())
print(lapsed['transaction_date'].max())

2015-04-28
2017-02-26


In [4]:
chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date']):
    chunk = chunk.drop_duplicates()
    chunk=chunk[chunk['transaction_date']>=str(lapsed_date_begin)]
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())
dftrans_before_20180922=dftrans_before_20180922.drop_duplicates()    

1 2019-09-19 18:32:56.017353
2 2019-09-19 18:33:43.507098
3 2019-09-19 18:34:24.407547
4 2019-09-19 18:35:14.731280
5 2019-09-19 18:36:04.744536
6 2019-09-19 18:36:49.704209
7 2019-09-19 18:37:39.331471
8 2019-09-19 18:38:35.679108
9 2019-09-19 18:39:38.963923
10 2019-09-19 18:40:59.869678
11 2019-09-19 18:42:09.363321
12 2019-09-19 18:43:31.126086
13 2019-09-19 18:45:31.192811
14 2019-09-19 18:48:21.043611
15 2019-09-19 18:51:08.203963


In [5]:
all_rewards_most_recent=dftrans_before_20180922.append(lapsed)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
print(all_rewards_most_recent.shape)

del chunk
del dftrans_before_20180922
gc.collect()

(22943190, 2)


126

In [6]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [7]:
# Before the end of 2019Q1           
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recursive_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2019,5,4)] # hard-code due to the 1st week of daily in item available
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']>datetime.date(2018,9,22)] # hard-code
historical_daily_df=historical_daily_df.sort_values("week_end_dt")
print(historical_daily_df.shape)
historical_daily_df.tail(3)

(20, 2)


,file_path,week_end_dt
23,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2019-01-26
24,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2019-02-02
25,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2019-02-09


In [8]:
prior_quarter_End

datetime.date(2019, 5, 4)

In [9]:
recent_complete_quarter_End

datetime.date(2019, 8, 3)

In [10]:
df_historical_daily_20180929_to_before_Q=historical_daily_df[historical_daily_df['week_end_dt']<=prior_quarter_End]
df_historical_daily_in_recent_Q=historical_daily_df[(historical_daily_df['week_end_dt']>prior_quarter_End) & (historical_daily_df['week_end_dt']<=recent_complete_quarter_End)]
print(df_historical_daily_20180929_to_before_Q.shape)
print(df_historical_daily_in_recent_Q.shape)


(20, 2)
(0, 2)


In [11]:
weekly_2019_daily_df=list(recursive_file_gen("/home/jian/BigLots/"))

weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2018" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2017" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "Historical" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "hist_daily_data_itemlevel_decompressed" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if ".txt" in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "aily" in x]
weekly_2019_daily_df=pd.DataFrame({"file_path":weekly_2019_daily_df})
weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])

weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
print(weekly_2019_daily_df.shape)
weekly_2019_daily_df.head(2)

weekly_2019_daily_df_in_recent_Q=weekly_2019_daily_df[(weekly_2019_daily_df['week_end_dt']>prior_quarter_End) & (weekly_2019_daily_df['week_end_dt']<=recent_complete_quarter_End)]
# Change to > prior_quarter_End for the next quarter
print(weekly_2019_daily_df_in_recent_Q.shape)


weekly_2019_daily_df_before_Q=weekly_2019_daily_df[weekly_2019_daily_df['week_end_dt']<=prior_quarter_End]
weekly_2019_daily_df_before_Q=weekly_2019_daily_df_before_Q[weekly_2019_daily_df_before_Q['week_end_dt']>df_historical_daily_20180929_to_before_Q['week_end_dt'].max()]
print(weekly_2019_daily_df_before_Q.shape) # Q1 should be 0, and Q2&after not


(37, 2)
(13, 2)
(12, 2)


In [12]:
weekly_2019_daily_df_in_recent_Q.shape

(13, 2)

In [13]:
weekly_2019_daily_df_before_Q.shape

(12, 2)

In [14]:
df_all_daily_files_20190922_before_quarter=df_historical_daily_20180929_to_before_Q.append(weekly_2019_daily_df_before_Q)

df_all_daily_files_in_recent_quarter=df_historical_daily_in_recent_Q.append(weekly_2019_daily_df_in_recent_Q)


In [15]:
print(df_all_daily_files_20190922_before_quarter.shape)
df_all_daily_files_20190922_before_quarter.sort_values("week_end_dt")

(32, 2)


,file_path,week_end_dt
7,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-09-29
8,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-06
9,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-13
10,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-20
11,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-10-27
12,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-03
13,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-10
14,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-17
15,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-11-24
26,/home/jian/BigLots/hist_daily_data_itemlevel_d...,2018-12-01


In [16]:
print(df_all_daily_files_in_recent_quarter.shape)
df_all_daily_files_in_recent_quarter.sort_values("week_end_dt")

(13, 2)


,file_path,week_end_dt
18,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-11
19,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-18
20,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-25
21,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-01
22,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-08
23,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-15
24,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-22
25,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-06-29
26,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-07-06
27,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-07-13


In [17]:
rew_df_after_20180929_before_quarter=pd.DataFrame()
rew_df_after_recent_quarter_only=pd.DataFrame()


for file in df_all_daily_files_20190922_before_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates().rename(columns={"transaction_dt":"transaction_date"})
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_20180929_before_quarter=rew_df_after_20180929_before_quarter.append(df)
    print(file,datetime.datetime.now())

for file in df_all_daily_files_in_recent_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates().rename(columns={"transaction_dt":"transaction_date"})
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_recent_quarter_only=rew_df_after_recent_quarter_only.append(df)
    print(file,datetime.datetime.now())


/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt 2019-09-19 19:19:55.041327
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt 2019-09-19 19:20:23.819438
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181013.txt 2019-09-19 19:20:44.940734
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181020.txt 2019-09-19 19:21:07.423439
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt 2019-09-19 19:21:31.074026
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt 2019-09-19 19:21:55.446960
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt 2019-09-19 19:22:21.620182
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt 2019-09-19 19:22:52.347074


In [18]:
all_rewards_most_recent=all_rewards_most_recent.append(rew_df_after_20180929_before_quarter)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date']

all_rewards_most_recent_After=all_rewards_most_recent[['customer_id_hashed','transaction_date']].append(rew_df_after_recent_quarter_only)
all_rewards_most_recent_After=all_rewards_most_recent_After.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent_After=all_rewards_most_recent_After.drop_duplicates(['customer_id_hashed']).rename(columns={"transaction_date":"transaction_date_after_the_Quarter"})

all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_After,on="customer_id_hashed",how="outer")
del all_rewards_most_recent_After

gc.collect()

79

In [19]:
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date_before_the_Quarter'].fillna("2099-12-31")
all_rewards_most_recent['transaction_date_before_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_before_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_pre_Quarter_End']=prior_quarter_End-all_rewards_most_recent['transaction_date_before_the_Quarter']
# Filled the na before with a later adte 2019-12-31


all_rewards_most_recent['transaction_date_after_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_after_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_recentt_Quarter_End']=recent_complete_quarter_End-all_rewards_most_recent['transaction_date_after_the_Quarter']
print(datetime.datetime.now())

2019-09-19 20:10:07.877288


In [20]:
all_rewards_most_recent['Month_to_pre_Quarter_End']=all_rewards_most_recent['Days_to_pre_Quarter_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent['Month_to_recent_Quarter_End']=all_rewards_most_recent['Days_to_recentt_Quarter_End'].apply(lambda x: x.days/(365.25/12))
print(datetime.datetime.now())

2019-09-19 20:20:10.826041


In [21]:
all_rewards_most_recent['Group_before_the_Quarter']=np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>12) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']<0,"NotAvailable_Before_Quarter","NaN")
                                                                    )
                                                           )
                                                   )
all_rewards_most_recent['Group_after_the_Quarter']=np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>12) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']<0,"NotAvailable_Before_Q4","NaN")
                                                                    )
                                                           )
                                                   )

In [22]:
all_rewards_most_recent['Group_before_the_Quarter'].unique().tolist()

['Active', 'Lapsed_13_48', 'WD_48+', 'NotAvailable_Before_Quarter']

In [23]:
all_rewards_most_recent['Group_after_the_Quarter'].unique()

array(['Active', 'Lapsed_13_48', 'WD_48+'], dtype=object)

In [24]:
RecentQuarter_sales_by_id=pd.DataFrame()

for file in df_all_daily_files_in_recent_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_sales=df.groupby(['customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales_recent_Quarter"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(["customer_id_hashed"])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans_recent_Quarter"})
    
    df=pd.merge(df_sales,df_trans,on="customer_id_hashed",how="outer")
    
    RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.append(df)
    print(file,datetime.datetime.now())
RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.groupby("customer_id_hashed")['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()
RecentQuarter_sales_by_id['Recent_Quarter_Shopping_Group']="Recent_Quarter_Shopped"

/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-11/MediaStormDailySales20190514-121223-405.txt 2019-09-19 20:21:39.086187
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-18/MediaStormDailySales20190521-111232-391.txt 2019-09-19 20:22:24.501161
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-25/MediaStormDailySales20190528-111211-863.txt 2019-09-19 20:23:23.030314
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-01/MediaStormDailySales20190604-111936-950.txt 2019-09-19 20:24:14.646822
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-08/MediaStormDailySales20190611-122013-442.txt 2019-09-19 20:25:08.316015
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-15/MediaStormDailySales20190618-112500-846.txt 2019-09-19 20:26:01.082324
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-22/MediaStormDailySales20190625-111927-257.txt 2019-09-19 20:26:52.911777
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-29/MediaStormDailySales20190702-113523-928.txt 2019-09-19 20:27:46

In [25]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,RecentQuarter_sales_by_id,on="customer_id_hashed",how="outer")
all_rewards_most_recent.shape

(28427573, 13)

In [28]:
Recent_Quarter_new_sign_ups_files=list(recursive_file_gen("/home/jian/BigLots/"))

Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "Master" in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2018" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2017" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2016" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if ".txt" in x]

Recent_Quarter_new_sign_ups_files_df=pd.DataFrame({"file_path":Recent_Quarter_new_sign_ups_files})
Recent_Quarter_new_sign_ups_files_df['week_end_dt']=Recent_Quarter_new_sign_ups_files_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']<=str(recent_complete_quarter_End)]
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']>str(prior_quarter_End)]
print(Recent_Quarter_new_sign_ups_files_df.shape)
Recent_Quarter_new_sign_ups_files_df.head(2)

(13, 2)


,file_path,week_end_dt
16,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-11
17,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-18


In [26]:
df_new_sign_the_gap="/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip"
df_new_sign_the_gap=pd.read_table(df_new_sign_the_gap,sep="|",usecols=['customer_id_hashed','sign_up_date'])

In [29]:
all_new_sign_ups=pd.DataFrame()
for file in Recent_Quarter_new_sign_ups_files_df['file_path'].unique().tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    all_new_sign_ups=all_new_sign_ups.append(df)
all_new_sign_ups=all_new_sign_ups.append(df_new_sign_the_gap)
all_new_sign_ups=all_new_sign_ups.sort_values(['customer_id_hashed','sign_up_date'])
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")
print(all_new_sign_ups.shape)

all_new_sign_ups['NewRewards']="Recent_Quarter_New_Sign_Ups"
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")

(1614908, 2)


In [30]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_new_sign_ups,on="customer_id_hashed",how="outer")
all_rewards_most_recent['NewRewards']=all_rewards_most_recent['NewRewards'].fillna("Old_Rewards_Members")

all_rewards_most_recent['sales_recent_Quarter']=all_rewards_most_recent['sales_recent_Quarter'].fillna(0)
all_rewards_most_recent['trans_recent_Quarter']=all_rewards_most_recent['trans_recent_Quarter'].fillna(0)

all_rewards_most_recent['Recent_Quarter_Shopping_Group']=all_rewards_most_recent['Recent_Quarter_Shopping_Group'].fillna("Recent_Quarter_No_Shop")

all_rewards_most_recent.shape


(28897174, 15)

In [51]:
all_rewards_most_recent['Group_before_the_Quarter']=all_rewards_most_recent['Group_before_the_Quarter'].fillna("nan")
all_rewards_most_recent['Group_after_the_Quarter']=all_rewards_most_recent['Group_after_the_Quarter'].fillna("nan")

output_id_count=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})
output_id_Q4_sales=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['sales_recent_Quarter','trans_recent_Quarter'].sum().reset_index()

output=pd.merge(output_id_count,output_id_Q4_sales,on=['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'],how="outer")


In [52]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Tracker_By_Migration_JT'

In [53]:
output['Recent_Quarteer_Sale_per_ID']=output['sales_recent_Quarter']/output['id_count']
output['Recent_Quarteer_Sale_per_ID']=output['Recent_Quarteer_Sale_per_ID'].apply(lambda x: np.round(x,2))

output['inclusion/exclusion']=np.where((output['Group_before_the_Quarter']!="NotAvailable_Before_Quarter") & (output['NewRewards']=="Recent_Quarter_New_Sign_Ups"),"Exclusion","Inclusion")


In [54]:
output.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Tracker_By_Migration_JT/BL_2019Q2_rewards_migration_output_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [55]:
all_rewards_most_recent.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Tracker_By_Migration_JT/BL_2019Q2_rewards_migration_rewards_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [ ]:

all_rewards_most_recent=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Tracker_By_Migration_JT/BL_2019Q2_rewards_migration_rewards_data_JL_"+str(datetime.datetime.now().date())+".csv",dtype=str)

# Add Q3 Information

In [59]:
this_ongoing_quarter_start_date=prior_quarter_End+datetime.timedelta(days=92)
print(this_ongoing_quarter_start_date)

2019-08-04


In [61]:
this_ongoing_quarter_sign_ups=list(recursive_file_gen("/home/jian/BigLots/"))

this_ongoing_quarter_sign_ups=[x for x in this_ongoing_quarter_sign_ups if "Master" in x]
this_ongoing_quarter_sign_ups=[x for x in this_ongoing_quarter_sign_ups if "2018" not in x]
this_ongoing_quarter_sign_ups=[x for x in this_ongoing_quarter_sign_ups if "2017" not in x]
this_ongoing_quarter_sign_ups=[x for x in this_ongoing_quarter_sign_ups if "2016" not in x]
this_ongoing_quarter_sign_ups=[x for x in this_ongoing_quarter_sign_ups if ".txt" in x]

this_ongoing_quarter_sign_ups_files_df=pd.DataFrame({"file_path":this_ongoing_quarter_sign_ups})
this_ongoing_quarter_sign_ups_files_df['week_end_dt']=this_ongoing_quarter_sign_ups_files_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
this_ongoing_quarter_sign_ups_files_df=this_ongoing_quarter_sign_ups_files_df[this_ongoing_quarter_sign_ups_files_df['week_end_dt']>=str(this_ongoing_quarter_start_date)]
# Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']>str(prior_quarter_End)]
print(this_ongoing_quarter_sign_ups_files_df.shape)
this_ongoing_quarter_sign_ups_files_df.head(2)

(6, 2)


,file_path,week_end_dt
29,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-10
30,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-17


In [62]:
ongoing_new_sign_up_df=pd.DataFrame()
for file in this_ongoing_quarter_sign_ups_files_df['file_path'].unique().tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed'])
    ongoing_new_sign_up_df=ongoing_new_sign_up_df.append(df)
ongoing_new_sign_up_df=ongoing_new_sign_up_df.drop_duplicates("customer_id_hashed")
print(ongoing_new_sign_up_df.shape)

ongoing_new_sign_up_df['Ongoing_NewRewards']="Ongoing_Quarter_New_Sign_Ups"


(715973, 1)


In [65]:
this_ongoing_quarter_POS=list(recursive_file_gen("/home/jian/BigLots/"))

this_ongoing_quarter_POS=[x for x in this_ongoing_quarter_POS if "daily" in x.lower()]
this_ongoing_quarter_POS=[x for x in this_ongoing_quarter_POS if "history" not in x.lower()]

this_ongoing_quarter_POS=[x for x in this_ongoing_quarter_POS if "2018" not in x]
this_ongoing_quarter_POS=[x for x in this_ongoing_quarter_POS if "2017" not in x]
this_ongoing_quarter_POS=[x for x in this_ongoing_quarter_POS if "2016" not in x]
this_ongoing_quarter_POS=[x for x in this_ongoing_quarter_POS if ".txt" in x]

this_ongoing_quarter_POS_df=pd.DataFrame({"file_path":this_ongoing_quarter_POS})
this_ongoing_quarter_POS_df['week_end_dt']=this_ongoing_quarter_POS_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
this_ongoing_quarter_POS_df=this_ongoing_quarter_POS_df[this_ongoing_quarter_POS_df['week_end_dt']>=str(this_ongoing_quarter_start_date)]
# Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']>str(prior_quarter_End)]
print(this_ongoing_quarter_POS_df.shape)
this_ongoing_quarter_POS_df.head(2)

(6, 2)


,file_path,week_end_dt
31,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-10
32,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-08-17


In [67]:
ongoing_quarter_sales_by_id=pd.DataFrame()
for file in this_ongoing_quarter_POS_df['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_sales=df.groupby(['customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales_ongoing_quarter"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(["customer_id_hashed"])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans_ongoing_quarter"})
    
    df=pd.merge(df_sales,df_trans,on="customer_id_hashed",how="outer")
    
    ongoing_quarter_sales_by_id=ongoing_quarter_sales_by_id.append(df)
    print(file,datetime.datetime.now())
ongoing_quarter_sales_by_id=ongoing_quarter_sales_by_id.groupby("customer_id_hashed")['sales_ongoing_quarter','trans_ongoing_quarter'].sum().reset_index()
ongoing_quarter_sales_by_id['Ongoing_Quarter_Shopping_Group']="Ongoing_Quarter_Shopped"


/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-10/MediaStormDailySales20190813-120541-143.txt 2019-09-19 21:34:42.558457
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-17/MediaStormDailySales20190820-113628-440.txt 2019-09-19 21:35:41.504397
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-24/MediaStormDailySales20190827-113630-114.txt 2019-09-19 21:36:41.860692
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-31/MediaStormDailySales20190903-113649-462.txt 2019-09-19 21:37:42.582549
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-07/MediaStormDailySales20190910-120643-163.txt 2019-09-19 21:38:28.123706
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-14/MediaStormDailySales20190917-110425-322.txt 2019-09-19 21:39:23.250496


In [76]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,ongoing_new_sign_up_df,on="customer_id_hashed",how="outer")
all_rewards_most_recent=pd.merge(all_rewards_most_recent,ongoing_quarter_sales_by_id,on="customer_id_hashed",how="outer")

In [77]:
all_rewards_most_recent.columns.tolist()

['customer_id_hashed',
 'transaction_date',
 'transaction_date_before_the_Quarter',
 'transaction_date_after_the_Quarter',
 'Days_to_pre_Quarter_End',
 'Days_to_recentt_Quarter_End',
 'Month_to_pre_Quarter_End',
 'Month_to_recent_Quarter_End',
 'Group_before_the_Quarter',
 'Group_after_the_Quarter',
 'sales_recent_Quarter',
 'trans_recent_Quarter',
 'Recent_Quarter_Shopping_Group',
 'sign_up_date',
 'NewRewards',
 'Ongoing_NewRewards',
 'sales_ongoing_quarter',
 'trans_ongoing_quarter',
 'Ongoing_Quarter_Shopping_Group']

In [84]:
gc.collect()

192

In [85]:
deminsion_cols=['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group','Ongoing_NewRewards','Ongoing_Quarter_Shopping_Group']
for col in deminsion_cols:
    all_rewards_most_recent[col]=all_rewards_most_recent[col].fillna('nan')
    print(datetime.datetime.now(),col)
    gc.collect()
    

2019-09-19 21:59:09.134788 Group_before_the_Quarter
2019-09-19 21:59:11.824368 Group_after_the_Quarter
2019-09-19 21:59:13.636540 NewRewards
2019-09-19 21:59:15.815097 Recent_Quarter_Shopping_Group
2019-09-19 21:59:20.508047 Ongoing_NewRewards
2019-09-19 21:59:22.868166 Ongoing_Quarter_Shopping_Group


In [87]:
output_everygroup.head(2)

,Group_before_the_Quarter,Group_after_the_Quarter,NewRewards,Recent_Quarter_Shopping_Group,Ongoing_NewRewards,Ongoing_Quarter_Shopping_Group,id_count,sales_recent_Quarter,trans_recent_Quarter,sales_ongoing_quarter,trans_ongoing_quarter,Recent_Quarteer_Sale_per_ID
0,Active,Active,Old_Rewards_Members,Recent_Quarter_No_Shop,Ongoing_Quarter_New_Sign_Ups,Ongoing_Quarter_Shopped,1,0.0,0.0,293.79,1.0,0.0
1,Active,Active,Old_Rewards_Members,Recent_Quarter_No_Shop,Ongoing_Quarter_New_Sign_Ups,nan,2,0.0,0.0,NaN,NaN,0.0


In [92]:
output_id_count_everygroup=all_rewards_most_recent.groupby(deminsion_cols)['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})
output_id_sales_everygroup=all_rewards_most_recent.groupby(deminsion_cols)['sales_recent_Quarter','trans_recent_Quarter','sales_ongoing_quarter','trans_ongoing_quarter'].sum().reset_index()

output_everygroup=pd.merge(output_id_count_everygroup,output_id_sales_everygroup,on=deminsion_cols,how="outer")


output_everygroup['Recent_Quarteer_Sale_per_ID']=output_everygroup['sales_recent_Quarter']/output_everygroup['id_count']
output_everygroup['Recent_Quarteer_Sale_per_ID']=output_everygroup['Recent_Quarteer_Sale_per_ID'].apply(lambda x: np.round(x,2))
output_everygroup['Recent_Quarteer_Trans_per_ID']=output_everygroup['trans_recent_Quarter']/output_everygroup['id_count']
output_everygroup['Recent_Quarteer_Trans_per_ID']=output_everygroup['Recent_Quarteer_Trans_per_ID'].apply(lambda x: np.round(x,2))

output_everygroup['ongoing_Quarteer_Sale_per_ID']=output_everygroup['sales_ongoing_quarter']/output_everygroup['id_count']
output_everygroup['ongoing_Quarteer_Sale_per_ID']=output_everygroup['ongoing_Quarteer_Sale_per_ID'].apply(lambda x: np.round(x,2))
output_everygroup['ongoing_Quarteer_Trans_per_ID']=output_everygroup['trans_ongoing_quarter']/output_everygroup['id_count']
output_everygroup['ongoing_Quarteer_Trans_per_ID']=output_everygroup['ongoing_Quarteer_Trans_per_ID'].apply(lambda x: np.round(x,2))


In [93]:
output_everygroup.shape

(36, 15)

In [94]:
output_everygroup=output_everygroup.rename(columns={"NewRewards":"Recent_Quarteer_NewRewards"})

In [95]:
output_everygroup.to_csv("./BL_2019Q3_tracking_migration_status_JL_"+str(datetime.datetime.now().date())+".csv",index=False)